### Monte Carlo - Black-Scholes - Merton - MSFT

In [1]:
import numpy as np
import pandas as pd 
from pandas_datareader import data as wb
import matplotlib.pyplot as plt 
from scipy.stats import norm
from datetime import datetime

In [3]:
from iexfinance.stocks import Stock, get_historical_data

start = datetime(2018, 1, 1)
end = datetime(2023, 3, 24)

api_key = 'API_KEY'

In [4]:
tickers = ['MSFT']
mydata = pd.DataFrame()
for t in tickers:
    mydata[t] = get_historical_data(t, start, end, output_format = 'pandas', token=api_key)['close']

$$
d_1 = \frac{\ln(\frac{S}{K}) + (r + \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

$$
d_2 = d_1 - s \cdot \sqrt{t} = \frac{\ln(\frac{S}{K}) + (r - \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

In [5]:
def d1(S, K, r, stdev,T):
    return (np.log(S/K) +  (r + stdev**2/2) * T)/(stdev * np.sqrt(T))

def d2(S, K, r, stdev,T):
    return (np.log(S/K) +  (r - stdev**2/2) * T)/(stdev * np.sqrt(T))

In [6]:
norm.cdf(0)

0.5

In [7]:
norm.cdf(0.25)

0.5987063256829237

In [20]:
norm.cdf(0.75)

0.7733726476231317

In [9]:
norm.cdf(9)

1.0

$$
\textbf{C} = SN(d_1) - Ke^{-rt}N(d_2) 
$$

In [10]:
def BSM(S, K, r, stdev,T):
    return (S * norm.cdf(d1(S, K, r, stdev, T))) - (K * np.exp(-r * T) * norm.cdf(d2(S, K, r, stdev, T)))

In [12]:
tickers = ['MSFT']
sec_data = pd.DataFrame()
for t in tickers:
    sec_data[t] = get_historical_data(t, start, end, output_format = 'pandas', token=api_key)['close']
    
sec_data = sec_data.astype('float64')

In [13]:
S = sec_data.iloc[-1]
S

MSFT    280.57
Name: 2023-03-24 00:00:00, dtype: float64

In [14]:
log_returns = np.log(1 + sec_data.pct_change())
log_returns

,MSFT
2018-01-02,NaN
2018-01-03,0.004643
2018-01-04,0.008763
2018-01-05,0.012322
2018-01-08,0.001020
...,...
2023-03-20,-0.026105
2023-03-21,0.005678
2023-03-22,-0.005457
2023-03-23,0.019530


In [15]:
stdev = log_returns.std()*250**0.5
stdev

MSFT    0.309515
dtype: float64

In [29]:
r = 0.0351
K = 300
T = 1

In [30]:
d1(S, K, r, stdev, T)

MSFT    0.051825
dtype: float64

In [31]:
d2(S, K, r, stdev, T)

MSFT   -0.25769
dtype: float64

In [32]:
BSM(S, K, r, stdev, T)

MSFT    30.707952
Name: 2023-03-24 00:00:00, dtype: float64